In [2]:
!pip install --upgrade google-cloud-vision google-cloud-language Pillow==4.0.0
!curl -L -o gdg.zip "https://drive.google.com/uc?export=download&id=13N61_erux7y9nTxJcyOdLyxlXKaH1lnn"
!unzip gdg.zip
from google.cloud import vision
from PIL import Image, ImageDraw

import io
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "gdg-cloudfest-keyfile.json"

Requirement already up-to-date: google-cloud-vision in /usr/local/lib/python3.6/dist-packages (0.35.0)
Requirement already up-to-date: google-cloud-language in /usr/local/lib/python3.6/dist-packages (1.1.0)
Requirement already up-to-date: Pillow==4.0.0 in /usr/local/lib/python3.6/dist-packages (4.0.0)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1010      0 --:--:-- --:--:-- --:--:--  1007
100  773k  100  773k    0     0  1376k      0 --:--:-- --:--:-- --:--:-- 1376k
Archive:  gdg.zip
replace gdg-cloudfest-keyfile.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: gdg-cloudfest-keyfile.json  
  inflating: images/car.jpg          
  inflating: images/car_and_person.jpg  
  inflating: images/faces.jpg        
  inflating: images/receipt.jpg      


In [3]:
# Let's create an Image Annotator Client object on Google Cloud
client = vision.ImageAnnotatorClient()
print(client)

In [0]:
def detect_license_plate(path):
    """Detects document features in an image."""
    

    # We open the file and save its contents to the variable content
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    # Also open the file in PIL so we can draw on it later
    source_img = Image.open(path)

    # Create a PIL draw object which will handle the actual drawing for us
    draw = ImageDraw.Draw(source_img)

    # We create an Image type from the image that the google cloud
    # library knows how to work with
    image = vision.types.Image(content=content)

    # Use the document text detector to search for text in the image
    # Fortunatly for us, a licence plate is text!
    response = client.document_text_detection(image=image)
    
    return response

In [5]:
print("="*20)
print(detect_license_plate(os.path.join('images', 'car.jpg')))
print("="*20)

text_annotations {
  locale: "en"
  description: "PENNSYLVANIA\nKBW-0714\nVISIPA.COM\n"
  bounding_poly {
    vertices {
      x: 766
      y: 491
    }
    vertices {
      x: 872
      y: 491
    }
    vertices {
      x: 872
      y: 543
    }
    vertices {
      x: 766
      y: 543
    }
  }
}
text_annotations {
  description: "PENNSYLVANIA"
  bounding_poly {
    vertices {
      x: 791
      y: 491
    }
    vertices {
      x: 847
      y: 492
    }
    vertices {
      x: 847
      y: 500
    }
    vertices {
      x: 791
      y: 499
    }
  }
}
text_annotations {
  description: "KBW"
  bounding_poly {
    vertices {
      x: 766
      y: 493
    }
    vertices {
      x: 808
      y: 493
    }
    vertices {
      x: 808
      y: 535
    }
    vertices {
      x: 766
      y: 535
    }
  }
}
text_annotations {
  description: "-"
  bounding_poly {
    vertices {
      x: 810
      y: 493
    }
    vertices {
      x: 821
      y: 493
    }
    vertices {
      x: 821
      y: 

In [0]:
def detect_license_plate(path):
    """Detects document features in an image."""
    

    # We open the file and save its contents to the variable content
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    # Also open the file in PIL so we can draw on it later
    source_img = Image.open(path)

    # Create a PIL draw object which will handle the actual drawing for us
    draw = ImageDraw.Draw(source_img)

    # We create an Image type from the image that the google cloud
    # library knows how to work with
    image = vision.types.Image(content=content)

    # Use the document text detector to search for text in the image
    # Fortunatly for us, a licence plate is text!
    response = client.document_text_detection(image=image)
    print("="*20)
    print(response)
    print("="*20)

    # The repsons contains all possible information the API was able to
    # gather from the image. A block represents a single piece of text it
    # was able to detect. A page is an aggregation of blocks
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            # For each block of text we can show the confidence the algortihtm
            # has in that particular block
            print('\nBlock confidence: {}\n'.format(block.confidence))

            # But more importantly: we need the bounding box around the object
            # so we can paint that rectangle black and obscure the license plate
            v = block.bounding_box.vertices
            print('\nVertices:\n{}'.format(v))

            # PIL only needs the top left and bottom right corners of the 
            # rectangle to draw it properly. These are the 1st and 3rd items
            # in the vertices. PIL also needs the coordinates to be in the
            # form of [x1, y1, x2, y2]
            box = [v[0].x, v[0].y, v[2].x, v[2].y]

            # And finally we draw the black box exactly where the license
            # plate was detected
            draw.rectangle(box, fill='black')

    # Save the image with the black bars on it
    base_path = os.path.basename(path)
    anon_path = os.path.splitext(base_path)[0]
    source_img.save(os.path.join("anon_images", "%s.jpg") % anon_path, "JPEG")

In [7]:
detect_license_plate(os.path.join("images", "car.jpg"))

text_annotations {
  locale: "en"
  description: "PENNSYLVANIA\nKBW-0714\nVISIPA.COM\n"
  bounding_poly {
    vertices {
      x: 766
      y: 491
    }
    vertices {
      x: 872
      y: 491
    }
    vertices {
      x: 872
      y: 543
    }
    vertices {
      x: 766
      y: 543
    }
  }
}
text_annotations {
  description: "PENNSYLVANIA"
  bounding_poly {
    vertices {
      x: 791
      y: 491
    }
    vertices {
      x: 847
      y: 492
    }
    vertices {
      x: 847
      y: 500
    }
    vertices {
      x: 791
      y: 499
    }
  }
}
text_annotations {
  description: "KBW"
  bounding_poly {
    vertices {
      x: 766
      y: 493
    }
    vertices {
      x: 808
      y: 493
    }
    vertices {
      x: 808
      y: 535
    }
    vertices {
      x: 766
      y: 535
    }
  }
}
text_annotations {
  description: "-"
  bounding_poly {
    vertices {
      x: 810
      y: 493
    }
    vertices {
      x: 821
      y: 493
    }
    vertices {
      x: 821
      y: 

In [0]:
def mask_faces(path):
    """Detects faces in an image."""

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    source_img = Image.open(path)
    draw = ImageDraw.Draw(source_img)

    image = vision.types.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations

    for face in faces:

        v = face.bounding_poly.vertices

        box = [v[0].x, v[0].y, v[2].x, v[2].y]

        draw.rectangle(box, fill='black')

    base_path = os.path.basename(path)
    anon_path = os.path.splitext(base_path)[0]
    source_img.save(os.path.join("anon_images", "%s.jpg") % anon_path, "JPEG")

In [0]:
mask_faces(os.path.join("images", "faces.jpg"))

In [0]:
from google.cloud import language

vision_client = vision.ImageAnnotatorClient()
language_client = language.LanguageServiceClient()

In [0]:
# Open image with PIL to draw on it
def mask_receipt(path):
    source_img = Image.open(path)
    draw = ImageDraw.Draw(source_img)

    # Call the vision API to extract text from the image
    with io.open(path, 'rb') as f:
        image = vision.types.Image(content=f.read())

    imageresponse = vision_client.document_text_detection(image=image)
    text = ' - '.join(imageresponse.text_annotations[0].description.split('\n'))
    
    print(text)

    # Call the natural language API to extract entities for all the text we found
    document = language.types.Document(content=text, type=language.enums.Document.Type.PLAIN_TEXT)
    languageresponse = language_client.analyze_entities(document=document)

    # Collect all text that was identified as a "PERSON"
    entities = languageresponse.entities
    persons = [e.name for e in entities if e.type == 1]

    # Loop over all persons and draw black boxes over them
    for person in persons:
        for annotation in imageresponse.text_annotations[1:]:
            if annotation.description in person:
                v = annotation.bounding_poly.vertices
                box = [v[0].x, v[0].y, v[2].x, v[2].y]
                draw.rectangle(box, fill='black')
    
    base_path = os.path.basename(path)
    anon_path = os.path.splitext(base_path)[0]
    source_img.save(os.path.join("anon_images", "%s.jpg") % anon_path, "JPEG")

In [12]:
mask_receipt(os.path.join("images", "receipt.jpg"))

Walmart - Save money. Live better. - Walmart - MANAGER GERALD TYLER - ( 207 ) 623 - 8223 - ST# 2046 OP# 00008202 TE# 18 TR# 00758 - GAIN SHEET 003700044870 1.87 X - GAIN SHEET 003700044870 - 1.87 X - GAIN SHEET 003700044870 - 1.87 X - MCCORMICK 005210009170 F 0.54 0 - MCCORMICK 005210009170 F 0.54 0 - SUBTOTAL 6.69 - COUPON 37000 053700012987 - 3.00-0 - COUPON 37000 053700012987 - 3.00-0 - COUPON 37000 053700012987 3.00-0 - COUPON 52100 055210010076 F 1.00-0 - COUPON 52100 055210010076 F 1.00-0 - SUBTOTAL 4.31- - TAX 1 6.000 % - 0.28 - TOTAL 4.03- - CASH TEND 4.03- - CHANGE DUE 0.00 - # ITEMS SOLD 5 - TC# 9480 6195 8863 6507 9438 - We gladly accept valid - manufacturer & internet coupons. - 03/23/11 21:22:05 - 
